# Artificial Intelligence
## Computer Assignment 3 - Naive bayes
### Amirali Raygan - 810197623
###### Using 2-days Grace

In this project we are going to design an assistant to recognize positive comments from the negative ones. In orther to do this we use Digikala sample comments to calculate basic probabilities.

## At First 
we try to make a simple naive bayes system and step by step upgrade the effeciency.
This codes are the simplest ones.

In [1]:
from __future__ import unicode_literals
from hazm import *
import pandas as pd
import numpy as np
import math

In [18]:
train_data = pd.read_csv("comment_train.csv")
p_recommendation = len(train_data[train_data['recommend'] == 'recommended']) / len(train_data['recommend'])
t_recommendation = len(train_data[train_data['recommend'] == 'recommended'])
n_recommendation = len(train_data[train_data['recommend'] == 'not_recommended'])

### Preprocessing
Before modeling the naive bayes we have to preprocess on the data. In this part we seperate all the words used in the train file and calculate the times each of them reapet in negative/positive comments. Then we calculate the probability of each word depend on wheter the comment is positive or negative.   
P(word | positive) and P(word | positive). This process was done for each word using in positive commends and negative ones. Seprately for title words and comment words.

In [17]:
###  Preprocessing Comments
#  Recommended Comments
positive_comment_words = []
for i in train_data[ train_data['recommend'] == 'recommended' ]['comment']:
    a = i.split(' ')
    for j in a:
        positive_comment_words.append(j)

total = len(positive_comment_words)
comment_words_prob = {}
for i in positive_comment_words:
    if i in comment_words_prob:
        comment_words_prob[i] += 1
    else:
        comment_words_prob[i] = 1

for i in comment_words_prob:
    comment_words_prob[i] = comment_words_prob[i] / (t_recommendation)


In [19]:
# Not Recommended Comements
negative_comment_words = []
for i in train_data[ train_data['recommend'] == 'not_recommended' ]['comment']:
    a = i.split(' ')
    for j in a:
        negative_comment_words.append(j)

total = len(negative_comment_words)
ncomment_words_prob = {}
for i in negative_comment_words:
    if i in ncomment_words_prob:
        ncomment_words_prob[i] += 1
    else:
        ncomment_words_prob[i] = 1

for i in ncomment_words_prob:
    ncomment_words_prob[i] = (ncomment_words_prob[i]) / (n_recommendation) 


In [21]:
#  Recommended Titles
positive_title_words = []
for i in train_data[ train_data['recommend'] == 'recommended' ]['title']:
    a = i.split(' ')
    for j in a:
        positive_title_words.append(j)

total = len(positive_title_words)
title_words_prob = {}
for i in positive_title_words:
    if i in title_words_prob:
        title_words_prob[i] += 1
    else:
        title_words_prob[i] = 1

for i in title_words_prob:
    title_words_prob[i] = title_words_prob[i] / (t_recommendation)


In [20]:
#  Not Recommended Titles
negative_title_words = []
for i in train_data[ train_data['recommend'] == 'recommended' ]['title']:
    a = i.split(' ')
    for j in a:
        negative_title_words.append(j)

total = len(negative_title_words)
ntitle_words_prob = {}
for i in negative_title_words:
    if i in ntitle_words_prob:
        ntitle_words_prob[i] += 1
    else:
        ntitle_words_prob[i] = 1

for i in ntitle_words_prob:
    ntitle_words_prob[i] = ntitle_words_prob[i] / (n_recommendation)


### Prediction phase
In this part we are trying to calculate the probability of each comment to be positive or negative depend on seeing each word.
According to bayes rule we have:                                 
P( positive , word) = P(Word | positive) * P(Positive) 

### Evidence 
The probability of each word in positive comment and in negative comment is our evidence. Also the probability of a comment to be positive or not is another evidence. Both of them can be calculated with train file.

### Liklihood
This contains the probablity of each comment to be positive or not.(recommended or not recommended) we calculate it with previous dictionary which store the probability of each word in train file.

### Prior
All of the probabilities which can be calculated by train file are our priors. 

### Posterior
This contains the probabilities that are questioned. Specificly we want to claculate the probability of each comment to be positive or negative.

In [22]:
### Prediction phase
test_data = pd.read_csv("comment_test.csv")
correct = 0
recom_correct = 0
recom_pred = 0
for j in range( 1, len( test_data ) ):
    comment_words = test_data['comment'][j].split(' ')
    recommend_prob = 0
    not_recommend_prob = 0
    for i in comment_words:
        if i in comment_words_prob:
            recommend_prob += math.log(comment_words_prob[i])
        else:
            recommend_prob -= 10000
        if i in ncomment_words_prob:
            not_recommend_prob += math.log(ncomment_words_prob[i])
        else:
            not_recommend_prob -= 10000
    
    title_words = test_data['title'][j].split(' ')
    for i in title_words:
        if i in title_words_prob:
            recommend_prob += math.log(title_words_prob[i])
        else:
            recommend_prob -= 10000
        if i in ntitle_words_prob:
            not_recommend_prob += math.log(ntitle_words_prob[i])
        else:
            not_recommend_prob -= 10000
        
    
    recommend_prob += math.log(p_recommendation)
    not_recommend_prob += math.log(1-p_recommendation)

    if recommend_prob > not_recommend_prob:
        recom_pred += 1
    if test_data['recommend'][j] == 'recommended' and recommend_prob > not_recommend_prob:
        correct += 1
        recom_correct += 1
    if test_data['recommend'][j] == 'not_recommended' and recommend_prob < not_recommend_prob:
        correct += 1

## Evaluation part

Here we have four types of evaluation.
### Accuracy
Accuracy gives us the totall correct predictions devided by all comments.

### Precision 
To calculating the precision, first we count the number of comments that predicted 'recommended' correctly and devide it to all the 'recommended' predicted comments. Obviously this is a evaluation type which give us the totall  efficiency becuase its only depend on one group of answers.

### Recall
Recall, correct 'recommended' detects devided by all actual 'recommended's. It's not valid on its own to. Excatly like the previuos one.

### F1
F1 is another measure of accuracy test. F1 score is the harmonic mean of the precision and recall.



Both Precision and recall are not enaugh for testing the program. For instance if there is a sample that contains a small number of 'recommended' prediction, these two values may become near to 1 but it doesnt mean that the accuracy of totall program is good. These two can express the accuracy of the program when its prediction is a special value.('recomended')

In [23]:
accuracy = correct / len(test_data)
precision = recom_correct / recom_pred
recall = recom_correct /  len(test_data[test_data['recommend'] == 'recommended'])
f1 = 2 * (precision*recall) / (precision+recall)
print("Accuracy =", accuracy)
print("Precision =", precision)
print("Recall =", recall)
print("F1 =", f1)

Accuracy = 0.89
Precision = 0.8513513513513513
Recall = 0.945
F1 = 0.8957345971563981


### More of Preprocesing
There are many tasks before the main process that can help us improve are accuracy. Hazm library provides us some of these. One of them are Normalizer function which normalize each string. By using this function our accuracy drops down to 0.88. So we decided not to use it. The other function is Stemmer which remove the prepositions and so on. THIS IS AWFUL. It reduce the accuracy down to 0.80 !! we do not use this either. The other one is lemmatizer which is little bit better than other functions. It returns the root of each verb instead of the exact verb.This function make a little difference on the accuracy which is about 0.1.


### Additive Smoothing
In some cases we have new words in our test file that we havent seen in train comments. When this happens the probability of new word become 0 and the whole probability of the comment goes to 0. To solve the problem we add some a constant number(=1) to each word probability to numerator and total number of positive/negative words to the denominator.

The most considerable change is when we add additive smoothing which improve our aaccuracy by 4%.

## Final code
Final program which provides us best accuracy is now here.
Lets take a look at the code.

In [33]:
from __future__ import unicode_literals
from hazm import *
import pandas as pd
import numpy as np
import math

train_data = pd.read_csv("comment_train.csv")
p_recommendation = len(train_data[train_data['recommend'] == 'recommended']) / len(train_data['recommend'])
t_recommendation = len(train_data[train_data['recommend'] == 'recommended'])
n_recommendation = len(train_data[train_data['recommend'] == 'not_recommended'])
###  Preprocessing Comments
#  Recommended Comments
normalizer = Normalizer()
stemmer = Stemmer()
lemmatizer = Lemmatizer()
positive_comment_words = []
for i in train_data[ train_data['recommend'] == 'recommended' ]['comment']:
    #a = normalizer.normalize(i)
    a = word_tokenize(i)
    for j in a:
        #positive_comment_words.append( stemmer.stem(j) )
        positive_comment_words.append( lemmatizer.lemmatize(j) )

total = len(positive_comment_words)
comment_words_prob = {}
for i in positive_comment_words:
    if i in comment_words_prob:
        comment_words_prob[i] += 1
    else:
        comment_words_prob[i] = 1

for i in comment_words_prob:
    comment_words_prob[i] = (comment_words_prob[i]+1) / (t_recommendation+total)

# Not Recommended Comemnets
negative_comment_words = []
for i in train_data[ train_data['recommend'] == 'not_recommended' ]['comment']:
    #a = normalizer.normalize(i)
    a = word_tokenize(i)
    for j in a:
        #negative_comment_words.append(stemmer.stem( j ) )
        negative_comment_words.append( lemmatizer.lemmatize(j) )

total = len(negative_comment_words)
ncomment_words_prob = {}
for i in negative_comment_words:
    if i in ncomment_words_prob:
        ncomment_words_prob[i] += 1
    else:
        ncomment_words_prob[i] = 1

for i in ncomment_words_prob:
    ncomment_words_prob[i] = (ncomment_words_prob[i]+1) / (n_recommendation+total) 
#  Recommended Titles
positive_title_words = []
for i in train_data[ train_data['recommend'] == 'recommended' ]['title']:
    #a = normalizer.normalize(i)
    a = word_tokenize(i)
    for j in a:
        #positive_title_words.append( stemmer.stem(j))
        positive_title_words.append( lemmatizer.lemmatize(j) )

total = len(positive_title_words)
title_words_prob = {}
for i in positive_title_words:
    if i in title_words_prob:
        title_words_prob[i] += 1
    else:
        title_words_prob[i] = 1

for i in title_words_prob:
    title_words_prob[i] = (title_words_prob[i]+1) / (t_recommendation +total)

#  Not Recommended Titles
negative_title_words = []
for i in train_data[ train_data['recommend'] == 'recommended' ]['title']:
    #a = normalizer.normalize(i)
    a = word_tokenize(i)
    for j in a:
        #negative_title_words.append( stemmer.stem(j) )
        negative_title_words.append( lemmatizer.lemmatize(j) )

total = len(negative_title_words)
ntitle_words_prob = {}
for i in negative_title_words:
    if i in ntitle_words_prob:
        ntitle_words_prob[i] += 1
    else:
        ntitle_words_prob[i] = 1

for i in ntitle_words_prob:
    ntitle_words_prob[i] = (ntitle_words_prob[i]+1) / (n_recommendation+total)


### Prediction Phase

test_data = pd.read_csv("comment_test.csv")
correct = 0
recom_correct = 0
recom_pred = 0
tt = []
cc = []
for j in range( 1, len( test_data ) ):
    ##comment_words = normalizer.normalize( test_data['comment'][j]) .split(' ')
    comment_words = word_tokenize(test_data['comment'][j])
    recommend_prob = 0
    not_recommend_prob = 0
    for i in comment_words:
        #st = stemmer.stem(i) 
        st = lemmatizer.lemmatize(i)
        if st in comment_words_prob:
            recommend_prob += math.log(comment_words_prob[st])
        else:
            recommend_prob -= 10000
        if st in ncomment_words_prob:
            not_recommend_prob += math.log(ncomment_words_prob[st])
        else:
            not_recommend_prob -= 10000
    
    #title_words = normalizer.normalize( test_data['title'][j] ).split(' ')
    title_words = test_data['title'][j]
    for i in title_words:
        #st = stemmer.stem(i) 
        st = lemmatizer.lemmatize(i)
        if st in title_words_prob:
            recommend_prob += math.log(title_words_prob[st])
        else:
            recommend_prob -= 10000
        if st in ntitle_words_prob:
            not_recommend_prob += math.log(ntitle_words_prob[st])
        else:
            not_recommend_prob -= 10000
    
    recommend_prob += math.log(p_recommendation)
    not_recommend_prob += math.log(1-p_recommendation)

    if recommend_prob > not_recommend_prob:
        recom_pred += 1
    if test_data['recommend'][j] == 'recommended' and recommend_prob > not_recommend_prob:
        correct += 1
        recom_correct += 1
    if test_data['recommend'][j] == 'not_recommended' and recommend_prob < not_recommend_prob:
        correct += 1
    
    if (test_data['recommend'][j] == 'not_recommended' and recommend_prob > not_recommend_prob ) or (test_data['recommend'][j] == 'recommended' and recommend_prob < not_recommend_prob):
        tt.append("Title = " + test_data['title'][j])
        cc.append("Comment = " + test_data['comment'][j])


accuracy = correct / len(test_data)
precision = recom_correct / recom_pred
recall = recom_correct /  len(test_data[test_data['recommend'] == 'recommended'])
f1 = 2 * (precision*recall) / (precision+recall)
print("Accuracy =", accuracy)
print("Precision =", precision)
print("Recall =", recall)
print("F1 =", f1)
print()

Accuracy = 0.91
Precision = 0.9
Recall = 0.9225
F1 = 0.9111111111111112



## Final Table
This part goes to describing the accuracy of additive smoothing and preprocessing functions. It is clear that additive smoothing is far more effective than preprocessing functions.

|           | a      | b      | c    | d     |
| --------- | ------ | ------ | ---- | ----- |
| Accuracy  | 0.9112 | 0.91   | 0.87 | 0.89  |
| Precision | 0.898  | 0.9    | 0.82 | 0.851 |
| Recall    | 0.927  | 0.9225 | 0.95 | 0.945 |
| F1        | 0.912  | 0.9111 | 0.88 | 0.895 |



#### Here are some comments that our program can not predict it correctly

In [32]:
for i in range(1,6):
    print(tt[i])
    print(cc[i])
    print()

Title = نقد پس از خرید
Comment = سلام ، راحت شدم از کابل شارژ ، توصیه میشود به شدت . ارزان گوشی خود را به شارژ وایرلس مجهز کنید .

Title = خیالم راحت شد
Comment = فندک قبلیم مدام فیوز میسوزوند و یک بار شارژر موبایل هم سوزوند ولی با این هیچ مشکلی بوجود نیومده تا الان. کیفیتش خیلی خوبه و لامپ هم داره

Title = محصول ضعیفی هست
Comment = بدون پخش بو و ماندگاری زیاد امیدوارم کیفیتش رو ارتقا بدن

Title = نخرید خوب نیس اصلا
Comment = با سلام، با چیزی که توی عکس هس فرق داره، اصلا روی گوشی درست واینمیسته، ی لبه گوشی خالی میمونه نمیرسه ب ال سی دی، من ک مرجوع کردم

Title = بررسی فیلتر سرکان
Comment = من خودم جزو افرادی بودم که نزدیک سیزده ساله از انواع فیلتر سرکان اعم از روغن، هوا و اتاق استفاده میکردم ولی به تازگی متوجه و اطلاع یافتم که فیلتر گاج باکیفیت تر از فیلتر سرکان می‌باشد و هم چنین قیمت بمراتب مناسب تربت نسبت به سرکان دارد و طرف فروشنده که داشت روغن فیلترش را به من می‌فروخت. واقعا بهم اثبات کرد که گاج باکیفیت تر از سرکان می‌باشد.



ambiguous words may cuase the wrong predictions. Although we have additive smoothing but new words may be a point for wrong predictions. 